In [15]:
#This class store the internsity and pitch 
class Pause_Info:
  def __init__(self,intensity,stinten,pitch,stpit,minsound,maxsound,performance,id,jokeName):
    self.intensity = intensity
    self.stinten= stinten
    self.pitch = pitch
    self.stpit= stpit
    self.minsound=minsound
    self.maxsound=maxsound
    self.performance=performance
    self.id=id
    self.jokeName=jokeName
    
    
#this function loop through the folder "joke_name_matching" and make a dictionary of 
#joke id and joke name for each performance
def getJokeNameForEachPerformance(folderPath):
    os.chdir(folderPath)
    
    jokeNameDict={}
    
    for file in glob.glob("*.txt"):
        idDict={}
        filepath=folderPath+'\\'+file
        f = open(filepath, "r")
        key=f.readline()
        key=key.replace("\x00","")
        key=key.replace("\n","")
#         print(key)
        content=f.read();
        f.close()
        content=content.replace("\x00","")
        content=content.replace("\n\n","\n")
        if not content:
            continue
        lines=content.split('\n')
        lines.pop(0)
        lines.pop(-1)
        for line in lines:
            jokename=line.split(':')[0]
            id=int(line.split(':')[1].split('.')[0].split('_')[1])
#             print(jokename,id)
            idDict[id]=jokename
        jokeNameDict[key]=idDict
    
    return jokeNameDict
            

#This function extract intensity and pitch info from txt file and put all info in to a dictionary
def readTxT(path):
    f = open(path, "r")
    line = f.readline()
    
    #store the joke title
    Title = line.split('\\')[-2]
    Title=Title.replace("\x00","")
    joke_dict = {}

    content=f.read();
    f.close()
    
    #replace all these werid \x00 in python
    #also replace all the extra newline char
    content=content.replace("\x00","")
    content=content.replace("\n\n","\n")
    lines=content.split('\n')
    
    #remove the first and last extra new line char
    lines.pop(0)
    lines.pop(-1)

    count=0
    intensity=0
    stinten=0
    pitch=0
    stpit=0
    minsound=0
    maxsound=0
    
    #store 7 info of each audio
    for line in lines:
        if(count%7==0):
            name=line.split('.')[0]
            name=name.split('_')[-1]
            id=int(name)
        elif(count%7==1):
            intensity=round(float(line))
        elif(count%7==2):
            stinten=round(float(line))
        elif(count%7==3):
            pitch=round(float(line))
        elif(count%7==4):
            stpit=round(float(line))
        elif(count%7==5):
            minsound=round(float(line))
        else:
            maxsound=round(float(line))
            if(Title not in jokeNameDict):
                break
            idDict=jokeNameDict[Title]
            jokeName=idDict[id]
            #print(jokeName)
            joke_dict[jokeName]=Pause_Info(intensity,stinten,pitch,stpit,minsound,maxsound,Title,id,jokeName)
           # print(Title,id)
        count+=1
    return joke_dict


import glob, os



#in combine_joke_dict,
#key of combine_joke_dict represent the joke id
#value of combine_joke_dict is an array of 'Pause_Info' who are the same joke but in different perofrmance
#len of value represent how many different perofrmance contain this joke 
def findAlltxtAndCombine(folderPath):
    os.chdir(folderPath)
    combine_joke_dict={}
    
    #find all txt file inside folder
    for file in glob.glob("*.txt"):
        filepath=folderPath+'\\'+file
        single_joke_dict=readTxT(filepath)   
        for x in single_joke_dict:
            y=single_joke_dict[x]
            if x in combine_joke_dict:
                combine_joke_dict[x].append(y)
            else:
                combine_joke_dict[x]=[y]
    return combine_joke_dict         

In [16]:
%matplotlib inline

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import scale
from numpy import random, float
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False

#info_arr represent an array of same joke in different performances
#choice=1 is using intensity and pitch as x-y axis
#choice=2 is using std intensity and std pitch as x-y axis
def clusterDataIn2D(Info_arr,choice):
    jokeid=Info_arr[0].jokeName.replace('/','-')
    data=[]
    x=[]
    y=[]
    performanceName=[]
    Xname,Yname='',''
    if(choice==1):
        for joke in Info_arr:
            X=joke.intensity
            Y=joke.pitch
            data.append((X,Y))
            performanceName.append(joke.performance)
            x.append(X)
            y.append(Y)
            Xname="intensity"
            Yname="pitch"
    elif(choice==2):
        for joke in Info_arr:
            X=joke.stinten
            Y=joke.stpit
            data.append((X,Y))
            performanceName.append(joke.performance)
            x.append(X)
            y.append(Y)
            Xname="standard deviation intensity"
            Yname="standard deviation pitch"
    
    #if the data set is smller than 2,we will ignore it.
    if(len(data)<2):
        return 
        
    # spilt them into 2 group
        
    model = KMeans(n_clusters=2)

    # Note I'm scaling the data to normalize it! Important for good results.
    model = model.fit(scale(data))

    # We can look at the clusters each data point was assigned to

    
    count=0
    Group0=[]
    Group1=[]
    for i in data:
        #print("performance Name: "+str(performanceName[count])+"\t","Data values: "+str(i)+"\t","Group ID: "+str(model.labels_[count]))  
        if(model.labels_[count]==0):
            Group0.append((performanceName[count],str(i)))
        else:
            Group1.append((performanceName[count],str(i)))
        count+=1

    dataTitle="Data values: ("+Xname+","+Yname+")\n"
    Group0txt="Group0\n"
    Group1txt="Group1\n"
    for a in Group0:
        Group0txt=Group0txt+"performance Name: "+str(a[0])+"  "+"Data values: "+str(a[1])+"\n"
        
    for a in Group1:
        Group1txt=Group1txt+"performance Name: "+str(a[0])+"  "+"Data values: "+str(a[1])+"\n"

    
    p=os.getcwd()
    
    
    #save all info into txt
    #print(jokeid)
    txtName="../jokeoutput/2D/"+str(jokeid)+".txt"

    if os.path.isfile(txtName):
        os.remove(txtName) 
    f = open(txtName, "a")
    f.write(dataTitle+Group0txt+Group1txt)
    f.close()
    
    print(jokeid)
    print (model.cluster_centers_)
    
    plt.figure(figsize=(10,6))
    plt.scatter(x, y, c=model.labels_.astype(float))
    plt.title("Joke Name: "+str(jokeid))
    plt.xlabel(Xname)
    plt.ylabel(Yname)
    
    pngName="../jokeoutput/2D/"+str(jokeid)+".png"
    
    #remove the existing png file
    if os.path.isfile(pngName):
        os.remove(pngName) 
    
    #save as png file
    plt.savefig(pngName)
    plt.show()
    

#this function will split data into 2 groups by using 4 features
def clusterDataIn4D(Info_arr):
    #replace all the '/' to '-' since it will mess up with path of windows
    jokeid=Info_arr[0].jokeName.replace('/','-')
    data=[]
    x=[]
    y=[]
    z=[]
    h=[]
    performanceName=[]
    Xname,Yname='',''
    
    #get all intensity,pitch,std intensity,std pitch
    for joke in Info_arr:
        X=joke.intensity
        Y=joke.pitch
        Z=joke.stinten
        H=joke.stpit
        data.append((X,Y,Z,H))
        performanceName.append(joke.performance)
        x.append(X)
        y.append(Y)
        z.append(Z)
        h.append(H)
        Xname="intensity"
        Yname="pitch"
        Zname="std intensity"
        Hname="std pitch"

    
    #if data set is smaller than 2, we cannot split them into 2groups
    if(len(data)<2):
        return 
        
    # spilt them into 2 group
        
    model = KMeans(n_clusters=2)

    # Note I'm scaling the data to normalize it! Important for good results.
    model = model.fit(scale(data))

    # We can look at the clusters each data point was assigned to

    
    count=0
    Group0=[]
    Group1=[]
    for i in data:
        #print("performance Name: "+str(performanceName[count])+"\t","Data values: "+str(i)+"\t","Group ID: "+str(model.labels_[count]))  
        if(model.labels_[count]==0):
            Group0.append((performanceName[count],str(i)))
        else:
            Group1.append((performanceName[count],str(i)))
        count+=1
    
    dataTitle="Data values: ("+Xname+","+Yname+","+Zname+","+Hname+")\n"
    Group0txt="Group0\n"
    Group1txt="Group1\n"
    for a in Group0:
        Group0txt=Group0txt+"performance Name: "+str(a[0])+"  "+"Data values: "+str(a[1])+"\n"
        
    for a in Group1:
        Group1txt=Group1txt+"performance Name: "+str(a[0])+"  "+"Data values: "+str(a[1])+"\n"

    
    p=os.getcwd()
    
    
    #save all info into txt
    txtName="../jokeoutput/4D/"+str(jokeid)+".txt"

    if os.path.isfile(txtName):
        os.remove(txtName) 
    f = open(txtName, "a")
    f.write(dataTitle+Group0txt+Group1txt)
    f.close()
    
    print(jokeid)
    print (model.cluster_centers_)


#this function will split data into 2 groups by using 6 features
def clusterDataIn6D(Info_arr):
    #replace all the '/' to '-' since it will mess up with path of windows
    jokeid=Info_arr[0].jokeName.replace('/','-')
    data=[]
    x=[]
    y=[]
    z=[]
    h=[]
    e=[]
    f=[]
    performanceName=[]
    Xname,Yname='',''
    
    #get all intensity,pitch,std intensity,std pitch
    for joke in Info_arr:
        X=joke.intensity
        Y=joke.pitch
        Z=joke.stinten
        H=joke.stpit
        E=joke.maxsound
        F=joke.minsound
        
        data.append((X,Y,Z,H,E,F))
        performanceName.append(joke.performance)
        x.append(X)
        y.append(Y)
        z.append(Z)
        h.append(H)
        e.append(E)
        f.append(F)
        
#         print(E,F)
        
        Xname="intensity"
        Yname="pitch"
        Zname="std intensity"
        Hname="std pitch"
        Ename="max sound"
        Fname="min sound"

    
    #if data set is smaller than 2, we cannot split them into 2groups
    if(len(data)<2):
        return 
        
    # spilt them into 2 group
        
    model = KMeans(n_clusters=2)

    # Note I'm scaling the data to normalize it! Important for good results.
    model = model.fit(scale(data))

    # We can look at the clusters each data point was assigned to

    
    count=0
    Group0=[]
    Group1=[]
    for i in data:
        #print("performance Name: "+str(performanceName[count])+"\t","Data values: "+str(i)+"\t","Group ID: "+str(model.labels_[count]))  
        if(model.labels_[count]==0):
            Group0.append((performanceName[count],str(i)))
        else:
            Group1.append((performanceName[count],str(i)))
        count+=1
    
    dataTitle="Data values: ("+Xname+","+Yname+","+Zname+","+Hname+","+Ename+","+Fname+")\n"
    Group0txt="Group0\n"
    Group1txt="Group1\n"
    for a in Group0:
        Group0txt=Group0txt+"performance Name: "+str(a[0])+"  "+"Data values: "+str(a[1])+"\n"
        
    for a in Group1:
        Group1txt=Group1txt+"performance Name: "+str(a[0])+"  "+"Data values: "+str(a[1])+"\n"

    
    p=os.getcwd()
    
    
    #save all info into txt
    txtName="../jokeoutput/6D/"+str(jokeid)+".txt"

    if os.path.isfile(txtName):
        os.remove(txtName) 
    f = open(txtName, "a")
    f.write(dataTitle+Group0txt+Group1txt)
    f.close()
    
    print(jokeid)
    print (model.cluster_centers_)


    
    
    
def main():
    
    global jokeNameDict
    jokeNameDict=getJokeNameForEachPerformance(r"E:\CapstoneWork\k-means-test_by_Tony\joke_name_matching")
    #i am using directory pitch_inten_avg_std_minmax since it is the test contain most information
    combine_joke_dict=findAlltxtAndCombine(r"E:\CapstoneWork\k-means-test_by_Tony\jokeinput")
    #print(len(combine_joke_dict))
#     for x in combine_joke_dict:
#         clusterDataIn2D(combine_joke_dict[x],1)
    for x in combine_joke_dict:
        clusterDataIn4D(combine_joke_dict[x])
    for x in combine_joke_dict:
        clusterDataIn6D(combine_joke_dict[x])
        
main()

robot_name_joke_2.ogg.wav
[[ 0.59861719  0.11076841 -0.67541655  0.06289643]
 [-1.09746484 -0.20307543  1.23826368 -0.11531013]]
gdpr_joke.ogg.wav
[[ 0.8405959   0.59462167 -0.76200076  0.49552012]
 [-0.8405959  -0.59462167  0.76200076 -0.49552012]]
tags-inverse_kinematics_equations_positive_2.ogg.wav
[[-0.70710678  0.64888568  0.70710678  0.50257071]
 [ 1.41421356 -1.29777137 -1.41421356 -1.00514142]]
family_joke.ogg.wav
[[ 0.80992387  0.72483372 -0.74318374  0.73880448]
 [-0.62994079 -0.56375956  0.5780318  -0.57462571]]
tags-family_joke_positive_2.ogg.wav
[[-0.66666667 -0.32084447  0.73782876 -0.11640505]
 [ 1.          0.48126671 -1.10674314  0.17460757]]
too_robotic_joke.ogg.wav
[[ 0.97058628  1.04512366 -1.01759796  0.96363636]
 [-0.5392146  -0.58062426  0.5653322  -0.53535354]]
c3po_joke.ogg.wav
[[ 0.92966968  0.77223504 -0.75482941  0.81016558]
 [-0.74373574 -0.61778803  0.60386353 -0.64813246]]
rosie_joke.ogg.wav
[[-0.75       -0.62308734  0.56920998 -0.62353829]
 [ 0.9375    

tags-inverse_kinematics_equations_positive_2.ogg.wav
[[-0.70710678  0.64888568  0.70710678  0.50257071  0.          0.        ]
 [ 1.41421356 -1.29777137 -1.41421356 -1.00514142  0.          0.        ]]
family_joke.ogg.wav
[[ 0.80992387  0.72483372 -0.74318374  0.73880448  0.          0.        ]
 [-0.62994079 -0.56375956  0.5780318  -0.57462571  0.          0.        ]]
tags-family_joke_positive_2.ogg.wav
[[-0.66666667 -0.32084447  0.73782876 -0.11640505  0.          0.        ]
 [ 1.          0.48126671 -1.10674314  0.17460757  0.          0.        ]]
too_robotic_joke.ogg.wav
[[-0.5392146  -0.58062426  0.5653322  -0.53535354  0.          0.        ]
 [ 0.97058628  1.04512366 -1.01759796  0.96363636  0.          0.        ]]
c3po_joke.ogg.wav
[[-0.74373574 -0.61778803  0.60386353 -0.64813246  0.          0.        ]
 [ 0.92966968  0.77223504 -0.75482941  0.81016558  0.          0.        ]]
rosie_joke.ogg.wav
[[ 0.9375      0.77885918 -0.71151247  0.77942286  0.          0.        ]